In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import copy
import random
import time


# --- 1. Definição do Problema e Parâmetros ---
SETUP_TIME = 15
DAILY_CAPACITY_MINUTES = 18 * 60 + 35

# Geração de dados simulados (mock data)
mock_data = [
    {'CODIGO_PRODUTO': '100010100023.0', 'Tinta': 'PRETO', 'Quantidade_Planejada': 2770, 'Estoque': 500, 'Pedidos': 2770, 'Data_de_Entrega': datetime.now() + timedelta(days=5), 'Tempo_Calculado': 1.1375},
    {'CODIGO_PRODUTO': '100010100024.0', 'Tinta': 'AZUL', 'Quantidade_Planejada': 2770, 'Estoque': 100, 'Pedidos': 2770, 'Data_de_Entrega': datetime.now() + timedelta(days=2), 'Tempo_Calculado': 1.0846},
    {'CODIGO_PRODUTO': '100010100025.0', 'Tinta': 'BRANCO', 'Quantidade_Planejada': 2770, 'Estoque': 3000, 'Pedidos': 2770, 'Data_de_Entrega': datetime.now() + timedelta(days=10), 'Tempo_Calculado': 2.6043},
    {'CODIGO_PRODUTO': '100010100026.0', 'Tinta': 'VERMELHO', 'Quantidade_Planejada': 1500, 'Estoque': 0, 'Pedidos': 1500, 'Data_de_Entrega': datetime.now() + timedelta(days=1), 'Tempo_Calculado': 0.4289},
    {'CODIGO_PRODUTO': '100010100027.0', 'Tinta': 'PRETO', 'Quantidade_Planejada': 1800, 'Estoque': 200, 'Pedidos': 1800, 'Data_de_Entrega': datetime.now() + timedelta(days=7), 'Tempo_Calculado': 0.7380},
    {'CODIGO_PRODUTO': '100010100028.0', 'Tinta': 'AMARELO', 'Quantidade_Planejada': 950, 'Estoque': 1000, 'Pedidos': 950, 'Data_de_Entrega': datetime.now() + timedelta(days=12), 'Tempo_Calculado': 0.4150},
    {'CODIGO_PRODUTO': '100010100029.0', 'Tinta': 'PRETO', 'Quantidade_Planejada': 300, 'Estoque': 0, 'Pedidos': 300, 'Data_de_Entrega': datetime.now() + timedelta(days=3), 'Tempo_Calculado': 0.1230},
    {'CODIGO_PRODUTO': '100010100030.0', 'Tinta': 'BRANCO', 'Quantidade_Planejada': 500, 'Estoque': 100, 'Pedidos': 500, 'Data_de_Entrega': datetime.now() + timedelta(days=4), 'Tempo_Calculado': 0.1980},
    {'CODIGO_PRODUTO': '100010100031.0', 'Tinta': 'AZUL', 'Quantidade_Planejada': 1200, 'Estoque': 0, 'Pedidos': 1200, 'Data_de_Entrega': datetime.now() + timedelta(days=2), 'Tempo_Calculado': 0.5220},
    {'CODIGO_PRODUTO': '100010100032.0', 'Tinta': 'AMARELO', 'Quantidade_Planejada': 220, 'Estoque': 500, 'Pedidos': 220, 'Data_de_Entrega': datetime.now() + timedelta(days=15), 'Tempo_Calculado': 0.0910},
]

cores = ['PRETO', 'AZUL', 'BRANCO', 'VERMELHO', 'AMARELO', 'VERDE', 'ROSA', 'CINZA', 'LARANJA', 'ROXO']
for i in range(33, 93):
    cor = random.choice(cores)
    qtd_planejada = random.randint(100, 3000)
    estoque = random.randint(0, 3000)
    pedidos = max(qtd_planejada - estoque, 0) if random.random() > 0.3 else qtd_planejada
    dias_entrega = random.randint(1, 20)
    tempo_calculado = round(random.uniform(0.1, 3.0), 4)
    
    mock_data.append({
        'CODIGO_PRODUTO': f'1000101000{i}.0', 'Tinta': cor, 'Quantidade_Planejada': qtd_planejada,
        'Estoque': estoque, 'Pedidos': pedidos, 'Data_de_Entrega': datetime.now() + timedelta(days=dias_entrega),
        'Tempo_Calculado': tempo_calculado
    })

# --- 2. Funções do Algoritmo ---

def calculate_cost(sequence):
    """Calcula o custo de setup de uma sequência com base nas trocas de cor."""
    total_setup_cost = 0
    if len(sequence) < 2:
        return 0
    for i in range(len(sequence) - 1):
        if sequence[i]['Tinta'] != sequence[i+1]['Tinta']:
            total_setup_cost += SETUP_TIME
    return total_setup_cost

def calculate_prioritization_score(item):
    """Calcula a pontuação de prioridade para um item."""
    saldo = item['Estoque'] - item['Pedidos']
    due_date = item['Data_de_Entrega']
    prioridade_urgente = 1 if saldo >= 0 else 0
    prioridade_data = due_date.timestamp()
    prioridade_saldo = abs(saldo) if saldo < 0 else 0
    return (prioridade_urgente, prioridade_data, -prioridade_saldo)

def daily_sequencing_and_scheduling(unscheduled_items):
    """Cria a programação inicial usando a heurística de priorização."""
    schedule = []
    day_number = 1
    remaining_items = copy.deepcopy(unscheduled_items)

    while remaining_items:
        remaining_items.sort(key=calculate_prioritization_score)
        
        items_for_today_prioritized = []
        time_used_today_minutes = 0
        items_to_remove_from_main_list = []
        
        last_color_of_day = None
        for item in remaining_items:
            item_production_time_minutes = item['Tempo_Calculado'] * 60
            setup_cost_to_add = 0
            
            if items_for_today_prioritized:
                if last_color_of_day != item['Tinta']:
                    setup_cost_to_add = SETUP_TIME

            if time_used_today_minutes + item_production_time_minutes + setup_cost_to_add <= DAILY_CAPACITY_MINUTES:
                items_for_today_prioritized.append(item)
                time_used_today_minutes += item_production_time_minutes
                if last_color_of_day != item['Tinta']:
                    time_used_today_minutes += setup_cost_to_add
                    last_color_of_day = item['Tinta']
                items_to_remove_from_main_list.append(item)
        
        if not items_for_today_prioritized:
            break
            
        color_groups = {}
        for item in items_for_today_prioritized:
            color = item['Tinta']
            if color not in color_groups:
                color_groups[color] = []
            color_groups[color].append(item)
        
        sorted_colors = sorted(color_groups.keys(), key=lambda c: calculate_prioritization_score(color_groups[c][0]))
        
        initial_daily_sequence = []
        for color in sorted_colors:
            initial_daily_sequence.extend(color_groups[color])

        current_day = {'day': day_number, 'items': initial_daily_sequence}
        schedule.append(current_day)
        
        remaining_items = [item for item in remaining_items if item not in items_to_remove_from_main_list]
        day_number += 1
        
    return schedule

# --- 3. Implementação da Busca Tabu ---

def tabu_search_optimizer(daily_sequence, tabu_tenure=7, max_iterations=100):
    """
    Otimiza a sequência de um único dia usando Busca Tabu para minimizar o custo de setup.
    """
    # Solução inicial e melhor solução encontrada
    current_solution = list(daily_sequence)
    best_solution = list(daily_sequence)
    best_cost = calculate_cost(best_solution)

    # A lista tabu armazena os "movimentos" proibidos. Um movimento é a troca de dois itens.
    tabu_list = []

    for iteration in range(max_iterations):
        best_neighbor = None
        best_neighbor_cost = float('inf')
        best_move = None

        # Explora a vizinhança da solução atual
        # A vizinhança é definida por todos os possíveis movimentos de troca (swap) de dois itens.
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                # Cria um vizinho trocando os itens i e j
                neighbor = list(current_solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                
                # O movimento é representado pelos índices trocados
                move = tuple(sorted((i, j)))

                # Calcula o custo do vizinho
                neighbor_cost = calculate_cost(neighbor)

                # Verifica se o movimento não é tabu e se é o melhor até agora
                if move not in tabu_list and neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost
                    best_move = move

        # Se encontrou um bom vizinho, atualiza a solução atual
        if best_neighbor:
            current_solution = best_neighbor
            
            # Adiciona o movimento à lista tabu
            tabu_list.append(best_move)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0) # Remove o movimento mais antigo
            
            # Atualiza a melhor solução global se necessário
            if best_neighbor_cost < best_cost:
                best_solution = best_neighbor
                best_cost = best_neighbor_cost
    
    return best_solution


# --- 4. Execução do Algoritmo Completo ---

if __name__ == "__main__":
    start_time = time.time()
    
    # 1. Gera a programação inicial, focada em prioridades
    initial_schedule = daily_sequencing_and_scheduling(mock_data)

    # 2. Refina cada dia da programação com a Busca Tabu, focado em minimizar setups
    optimized_schedule = []
    print("Otimizando a sequência de cada dia com a Busca Tabu...")
    for day_data in initial_schedule:
        initial_daily_sequence = day_data['items']
        
        # Aplica a otimização
        refined_daily_sequence = tabu_search_optimizer(initial_daily_sequence)
        
        # Calcula os novos custos e tempos
        refined_setup_cost = calculate_cost(refined_daily_sequence)
        production_time = sum(item['Tempo_Calculado'] * 60 for item in refined_daily_sequence)
        
        optimized_schedule.append({
            'day': day_data['day'],
            'items': refined_daily_sequence,
            'time_used_minutes': production_time + refined_setup_cost,
            'total_components': sum(item['Quantidade_Planejada'] for item in refined_daily_sequence)
        })
    print("Otimização concluída.")

    total_setup_optimized = sum(calculate_cost(day['items']) for day in optimized_schedule)
    total_production_time_optimized = sum(day['time_used_minutes'] for day in optimized_schedule)
    end_time = time.time()

    # Impressão do cronograma final
    print("\n" + "="*50 + "\n")
    print("Gerando o Cronograma de Produção Diário (Otimizado com Busca Tabu):")
    for day in optimized_schedule:
        print(f"\n--- Dia {day['day']} ---")
        print(f"Tempo Total de Trabalho: {day['time_used_minutes']:.2f} minutos ({day['time_used_minutes'] / 60:.2f} horas)")
        print(f"Total de Componentes Produzidos: {day['total_components']}")
        print("Itens a Serem Pintados (sequência otimizada):")
        for item in day['items']:
            saldo = item['Estoque'] - item['Pedidos']
            status = "URGENTE" if saldo < 0 else "normal"
            print(f"  - Tinta: {item['Tinta']}, Qtd: {item['Quantidade_Planejada']}, Saldo: {saldo} ({status}), Entrega: {item['Data_de_Entrega'].strftime('%Y-%m-%d')}")
            
    print("\n" + "="*50 + "\n")
    print("Resumo Semanal do Agendamento Otimizado:")
    print(f"Custo total de setup na semana: {total_setup_optimized} minutos")
    print(f"Tempo total de trabalho na semana: {total_production_time_optimized:.2f} minutos ({total_production_time_optimized / 60:.2f} horas)")
    print(f"Total de dias necessários: {len(optimized_schedule)}")
    print(f"Tempo de execução do algoritmo: {end_time - start_time:.4f} segundos")

Otimizando a sequência de cada dia com a Busca Tabu...
Otimização concluída.


Gerando o Cronograma de Produção Diário (Otimizado com Busca Tabu):

--- Dia 1 ---
Tempo Total de Trabalho: 1084.23 minutos (18.07 horas)
Total de Componentes Produzidos: 25792
Itens a Serem Pintados (sequência otimizada):
  - Tinta: VERMELHO, Qtd: 1500, Saldo: -1500 (URGENTE), Entrega: 2025-08-09
  - Tinta: AZUL, Qtd: 2770, Saldo: -2670 (URGENTE), Entrega: 2025-08-10
  - Tinta: AZUL, Qtd: 1200, Saldo: -1200 (URGENTE), Entrega: 2025-08-10
  - Tinta: VERDE, Qtd: 1273, Saldo: -120 (URGENTE), Entrega: 2025-08-10
  - Tinta: PRETO, Qtd: 300, Saldo: -300 (URGENTE), Entrega: 2025-08-11
  - Tinta: PRETO, Qtd: 2678, Saldo: -968 (URGENTE), Entrega: 2025-08-12
  - Tinta: PRETO, Qtd: 2770, Saldo: -2270 (URGENTE), Entrega: 2025-08-13
  - Tinta: PRETO, Qtd: 1800, Saldo: -1600 (URGENTE), Entrega: 2025-08-15
  - Tinta: PRETO, Qtd: 1641, Saldo: -645 (URGENTE), Entrega: 2025-08-16
  - Tinta: AMARELO, Qtd: 1035, Saldo: -1031 (